In [16]:
import pandas as pd
import simpledorff
import numpy as np
import operator

In [53]:
posts = pd.read_excel(f'xxxx/verify.xlsx')
lda = pd.read_excel(f'xxxx/post-membership.xlsx')

df = pd.merge(posts, lda, on='Id', how='inner')

df = df[['Id','Topic','Topic 1 Perc Contrib',
'Topic 2 Perc Contrib','Topic 3 Perc Contrib',
'Topic 4 Perc Contrib','Topic 5 Perc Contrib','Topic 6 Perc Contrib']].rename(columns={'Id':'Id','Topic':'Topic',
'Topic 1 Perc Contrib':'Coding','Topic 2 Perc Contrib':'Energy',
'Topic 3 Perc Contrib':'Device','Topic 4 Perc Contrib':'Monitoring',
'Topic 5 Perc Contrib':'CS','Topic 6 Perc Contrib':'DT'})

data = pd.DataFrame()
for i in range(len(posts.index)):
    aTopic = str(posts['Topic'][i]).lower().split(';')   
    aTopic = list(dict.fromkeys(aTopic))
    # thrs = 0.318800002336502102
    thrs = 0.348100006

    if df.loc[i]['Coding']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'coding'},index=[0])], ignore_index=True)
    if df.loc[i]['Energy']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'energy'},index=[0])], ignore_index=True)
    if df.loc[i]['Device']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'device'},index=[0])], ignore_index=True)
    if df.loc[i]['Monitoring']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'monitoring'},index=[0])], ignore_index=True)
    if df.loc[i]['CS']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'computing resource'},index=[0])], ignore_index=True)
    if df.loc[i]['DT']>=thrs: data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':'data transmission'},index=[0])], ignore_index=True)

    if df.loc[i]['Coding'] < thrs and df.loc[i]['Energy'] < thrs and df.loc[i]['Device'] < thrs and df.loc[i]['Monitoring'] < thrs and df.loc[i]['CS'] < thrs and df.loc[i]['DT'] < thrs:
        item = {'Coding':df.loc[i]['Coding'],'Energy': df.loc[i]['Energy'],'Device': df.loc[i]['Device'],'Monitoring': df.loc[i]['Monitoring'],
        'CS': df.loc[i]['CS'],'DT': df.loc[i]['DT'],}
        data = pd.concat([data,pd.DataFrame({'post_id':posts['Id'][i],
        'annotator_id':'LDA','annotation':max(item.items(), key=operator.itemgetter(1))[0]},index=[0])], ignore_index=True)
    
    for at in aTopic:
        if at in list(data[data['post_id']==posts['Id'][i]]['annotation']):
            temp = pd.DataFrame({'post_id':posts['Id'][i],
            'annotator_id':'Human','annotation':at},index=[0])
            data = pd.concat([data,temp], ignore_index=True)
len(data[data['annotator_id']=='LDA'])
# data  
simpledorff.calculate_krippendorffs_alpha_for_df(data,
experiment_col='post_id',
annotator_col='annotator_id',
class_col='annotation')

0.794949494949495